In [1]:
%load_ext autoreload
%autoreload 2

import asyncio
import nest_asyncio
import os
import sys
from typing import ClassVar, Optional, Any, Dict, List, Union

import anthropic
from dotenv import load_dotenv
import openai
import pandas as pd
import pydantic

sys.path.append('../')
from lattereview.providers.openai_provider import OpenAIProvider
from lattereview.agents.reviewer_agent import ReviewerAgent

# Load environment variables from .env file
load_dotenv('../.env')
print(os.getenv('OPENAI_API_KEY'))

# Enable asyncio in Jupyter
nest_asyncio.apply()

Failed to update token costs. Using static costs.


sk-cq_M0pNgHhCFnlDOCMnagYA1l2X7Yea5CL0ci5pZMNT3BlbkFJ0m0x9wm5M_EstX5SjLu_kdwGMDYjkUdviNPs4pe9cA


/Users/pouria/Documents/Coding/LatteReview/venv/lib/python3.9/site-packages/tokencost/constants.py:69: RuntimeWarning: coroutine 'update_token_costs' was never awaited
  logger.error("Failed to update token costs. Using static costs.")


In [3]:
df = pd.read_excel('data.xlsx')
df.head()

,PMID,PMC ID,Title,Author(s),Author(s) Full Name,Author(s) Affiliation,Corporate Author,Collaborator(s),Collaborator(s) Full Name,Collaborator(s) Affiliation,...,Manuscript Identifier,Secondary Source ID,Publishing Model,Comment on,Comment in,Erratum in,Erratum for,Corrected and Republished in,Corrected and Republished from,Owner
0,39618885,PMC11608428,Learning to Generate Context-Sensitive Backcha...,Bilalpur M||Inan M||Zeinali D||Cohn JF||Alikha...,"Bilalpur, Maneesh||Inan, Mert||Zeinali, Dorsa|...","University of Pittsburgh, Pittsburgh, Pennsylv...",NaN,NaN,NaN,NaN,...,NIHMS2038099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLM
1,39596657,PMC11593565,Assessing the Impact and Cost-Effectiveness of...,Liu SH||Weber ES||Manz KE||McCarthy KJ||Chen Y...,"Liu, Shelley H||Weber, Ellerie S||Manz, Kather...",Department of Population Health Science and Po...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLM
2,39594314,PMC11591420,Bridging Employees' Perceptions of Corporate S...,Wang Z||Carroll S||Wang EH,"Wang, Zheni||Carroll, Steve||Wang, Eric H",Department of Management/International-Busines...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLM
3,39531013,NaN,The influence of active agent motility on SIRS...,Kailasham R||Khair AS,"Kailasham, R||Khair, Aditya S","Department of Chemical Engineering, Carnegie M...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLM
4,39527845,NaN,Coupled infectious disease and behavior dynami...,Reitenbach A||Sartori F||Banisch S||Golovin A|...,"Reitenbach, Andreas||Sartori, Fabio||Banisch, ...","Sociology and Computational Social Science, Ka...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLM


In [9]:
openanai_provider = OpenAIProvider()
question = "What is the capital of France?"
asyncio.run(openanai_provider.get_response(question, temperature=0.9))

('The capital of France is Paris.',
 {'input_cost': 1.05e-06, 'output_cost': 4.2e-06, 'total_cost': 5.25e-06})

In [10]:
class ReviewerResponseFormat(pydantic.BaseModel):
    reasoning: str
    score: int
    need_help: bool 

agent = ReviewerAgent(
    provider=OpenAIProvider(output_json_format=ReviewerResponseFormat),
    name="Reviewer1",
    backstory="an expert reviewer and researcher!",
    temperature=0.5,
    max_tokens=150,
    review_type='title',
    review_criteria="Check if the title contains the word 'learning'",
    scoring_schema='[0, 1]',
    scoring_rules='Your scores should be either 0 or 1. 0 means that the paper does not meet the criteria, and 1 means that the paper meets the criteria.',
    help_criteria='Seek help when you are extremely uncertain about a paper.'
)

text_list = df.Title.str.lower().tolist()
asyncio.run(agent.review_items(text_list[:3]))

[('{"reasoning":"The article title includes the word \'learning\', which is the specific criterion that needs to be met. Therefore, it satisfies the requirement of containing the word \'learning\'.","score":1,"need_help":false}',
  {'input_cost': 2.655e-05, 'output_cost': 2.82e-05, 'total_cost': 5.475e-05}),
 ('{"reasoning":"The article title does not contain the word \'learning\'. Therefore, it does not meet the specified criteria.","score":0,"need_help":false}',
  {'input_cost': 2.85e-05, 'output_cost': 2.04e-05, 'total_cost': 4.89e-05}),
 ('{"reasoning":"The article title does not contain the word \'learning\'. Therefore, it does not meet the specified criteria.","score":0,"need_help":false}',
  {'input_cost': 2.73e-05, 'output_cost': 2.04e-05, 'total_cost': 4.77e-05})]

In [ ]:
text_list[:3]

['learning to generate context-sensitive backchannel smiles for embodied ai agents with applications in mental health dialogues.',
 "assessing the impact and cost-effectiveness of exposome interventions on alzheimer's disease: a review of agent-based modeling and other data science methods for causal inference.",
 "bridging employees' perceptions of corporate social responsibility, sense-making for meaningfulness, and work engagement for successful self-regulation."]